In [0]:
!nvidia-smi

## **Setup Detectron2**

In [0]:
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version

In [0]:
## As of the Feb/March 2020, this notebook works with the following version:
'''
detectron2-0.1+cu100-cp36-cp36m-linux_x86_64.whl
detectron2-0.1+cu100-cp37-cp37m-linux_x86_64.whl
detectron2-0.1+cu100-cp38-cp38-linux_x86_64.whl
'''

!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
import random
import os
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

## **Inference on pre-trained model (Detectron2 Model Zoo only!)**

In [0]:
im = cv2.imread("drive/My Drive/data/detectron/tolpum_0-15_160.jpg")
cv2_imshow(im)

In [0]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [0]:
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

## **Register custom dataset**

Here we're registering a dataset with COCO format (Exported from CVAT), but we could've have use other formats as well. 

See https://detectron2.readthedocs.io/tutorials/datasets.html#

In [0]:
!tar -xzvf "drive/My Drive/data/detectron/dataset.tar.gz"

register_coco_instances("train_dataset", {}, "./Detectron2/train/train.json", "./Detectron2/train")
train_metadata = MetadataCatalog.get("train_dataset")
train_dicts = DatasetCatalog.get("train_dataset")

register_coco_instances("test_dataset", {}, "./Detectron2/test/test.json", "./Detectron2/test")
test_metadata = MetadataCatalog.get("test_dataset")
test_dicts = DatasetCatalog.get("test_dataset")

## **Train custom dataset**

First, we visualize our labeled images to make sure they are loaded correct. 

Afterwards, change the parameters if needed before training.

In [0]:
for d in random.sample(train_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [0]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train_dataset",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 2000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [0]:
%load_ext tensorboard
%tensorboard --logdir output

## **Inference & Evaluation on custom dataset**

**IMPORTANT:** Run this section only if the previous sections (i.e. TRAIN) were run.

In [0]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.DATASETS.TEST = ("test_dataset", )
predictor = DefaultPredictor(cfg)

In [0]:
for d in random.sample(test_dicts, 10):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=test_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

In [0]:
evaluator = COCOEvaluator("test_dataset", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "test_dataset")
inference_on_dataset(trainer.model, val_loader, evaluator)

In [0]:
# SAVE MODER FOR LATER USE
# Optionally, save the Config file as well
!cp -r output/ "drive/My Drive/data/detectron/"

## **Inference on pre-trained model (Custom trained model)**

First of all, you need to make sure you use the same config file that was used for training (though not all parameters are actually needed). Also, remember to use the same classes names and ID's. 


Here we have a couple of options to run inference:

1. Register a new dataset just like we did for training with the only difference being that you don't need to have annotations since we want to run inference on unlabeled data.
2. Act as if you were registering a dataset by setting 'Metadata' only without actually creating a dataset. Seems like a 'hacky' solution but it is easier.

In [0]:
### 1

register_coco_instances("sample_dataset", {}, "drive/My Drive/data/detectron/sample/test.json", "drive/My Drive/data/detectron/sample/")
sample_metadata = MetadataCatalog.get("sample_dataset")
sample_dicts = DatasetCatalog.get("sample_dataset")

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.DATASETS.TEST = ("sample_dataset", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# !cp -r "drive/My Drive/data/detectron/output" .
cfg.MODEL.WEIGHTS = 'output/model_final.pth'

predictor = DefaultPredictor(cfg)

for d in sample_dicts:
  im = cv2.imread(d['file_name'])
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=sample_metadata, 
                scale=0.8, 
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
                 )
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(v.get_image()[:, :, ::-1])

In [0]:
### 2

## Get checkpoints / weights
# !cp -r "drive/My Drive/data/detectron/output" .

custom_metadata = MetadataCatalog.get('__unused')
custom_metadata.thing_classes = ['Pumas', 'Referee', 'Toluca']
custom_metadata.thing_dataset_id_to_contiguous_id = {1: 0, 2: 1, 3: 2}
custom_metadata.thing_colors = [(235, 64, 52), (235, 208, 52), (52, 235, 98)]

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = 'output/model_final.pth'

predictor = DefaultPredictor(cfg)

images = ['drive/My Drive/data/detectron/sample/tolpum_15-30_004.jpg',
          'drive/My Drive/data/detectron/sample/tolpum_15-30_005.jpg',
          'drive/My Drive/data/detectron/sample/tolpum_15-30_006.jpg',
          'drive/My Drive/data/detectron/sample/tolpum_15-30_007.jpg']

for d in images:
  im = cv2.imread(d)
  outputs = predictor(im)
  
  ## Save bboxes per image on a numpy file
  pred_boxes = outputs["instances"].to("cpu").pred_boxes
  f_out = os.path.basename(d).split('.')[0]
  np.save(f_out, pred_boxes.tensor.numpy())

  ## Visualize bboxes
#   v = Visualizer(im[:, :, ::-1],
#                 metadata=custom_metadata, 
#                 scale=0.8, 
#                 instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
#                  )
#   v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#   cv2_imshow(v.get_image()[:, :, ::-1])


In [0]:
## TESTING - IGNORE

pred_boxes = outputs["instances"].to("cpu").pred_boxes
scores = outputs["instances"].to("cpu").scores
pred_classes = outputs["instances"].to("cpu").pred_classes
pred_masks = outputs["instances"].to("cpu").pred_masks

In [0]:
## TESTING - IGNORE

# x0, y0, x1, y1 = pred_boxes[0].tensor.numpy()[0]
# width = x1 - x0
# height = y1 - y0

# print(x0, y0, x1, y1)

pred_boxes.tensor.numpy()

## **Inference with Custom Visualizer**

First we need to install Detectron2 from source.

In [0]:
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version

In [0]:
# This notebooks works with Commit # - 6ebb36af77543379fc56ce0287ca23ecaaefb7d8
!git clone https://github.com/facebookresearch/detectron2.git
%cd detectron2
!pip install -e .

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

## Get weights/checkpoint
!cp -r "/content/drive/My Drive/data/detectron/checkpoint_full" /content/
!cp "/content/drive/My Drive/data/detectron/custom_visualizer.py" /content/detectron2/detectron2/utils/

In [0]:
# detectron2/detectron2
%cd detectron2

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import pickle
import numpy as np
import cv2
import random
import os
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
# from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.utils.custom_visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

In [0]:
### 2

custom_metadata = MetadataCatalog.get('__unused')
custom_metadata.thing_classes = ['CenterCircle', 'PenaltyArea', 'Pumas', 'Referee', 'Toluca']
custom_metadata.thing_dataset_id_to_contiguous_id = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4}
custom_metadata.thing_colors = [(83, 237, 127), (83, 237, 127), (52, 235, 98), (50, 168, 82), (219, 245, 51)]

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = '/content/checkpoint_full/model_final.pth'

predictor = DefaultPredictor(cfg)

images = ['/content/drive/My Drive/data/detectron/tolpum_0-15_160.jpg']
for d in images:
  im = cv2.imread(d)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=custom_metadata, 
                # scale=0.8, 
                instance_mode=ColorMode.SEGMENTATION # IMAGE - SEGMENTATION - IMAGE_BW
                 )
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(v.get_image()[:, :, ::-1])


In [0]:
!cp /content/detectron2/detectron2/utils/custom_visualizer.py "/content/drive/My Drive/data/detectron/custom_visualizer.py"

In [0]:
## VIDEO

custom_metadata = MetadataCatalog.get('__unused')
custom_metadata.thing_classes = ['CenterCircle', 'PenaltyArea', 'Pumas', 'Referee', 'Toluca']
custom_metadata.thing_dataset_id_to_contiguous_id = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4}
custom_metadata.thing_colors = [(83, 237, 127), (83, 237, 127), (52, 235, 98), (50, 168, 82), (219, 245, 51)]

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = '/content/checkpoint_full/model_final.pth'

predictor = DefaultPredictor(cfg)

input_video_filename = '/content/drive/My Drive/data/detectron/pose_estimation.mp4'
output_video_filename = '/content/full_detections.mp4'

input_video = cv2.VideoCapture(input_video_filename)
width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = input_video.get(cv2.CAP_PROP_FPS)
num_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

output_video = cv2.VideoWriter(filename=output_video_filename,
                              fourcc=cv2.VideoWriter_fourcc(*'mp4v'),
                              fps=float(frames_per_second),
                              frameSize=(width, height),
                              isColor=True)

i = 0
while(input_video.isOpened()):

  if i % 60 == 0:
    prog = round(i / num_frames, 2) * 100
    print('{}%'.format(prog))

  ret, frame = input_video.read()
  if ret == True:
    
    outputs = predictor(frame)
    v = Visualizer(frame[:, :, ::-1],
                   metadata=custom_metadata, 
                   # scale=0.8, 
                   instance_mode=ColorMode.SEGMENTATION
                   )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    out = v.get_image()
    out = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
    output_video.write(out)

    i += 1
  else:
    break

input_video.release()
output_video.release()

In [0]:
## Save detections without visualizer

custom_metadata = MetadataCatalog.get('__unused')
custom_metadata.thing_classes = ['CenterCircle', 'PenaltyArea', 'Pumas', 'Referee', 'Toluca']
custom_metadata.thing_dataset_id_to_contiguous_id = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4}
custom_metadata.thing_colors = [(235, 64, 52), (235, 208, 52), (52, 235, 98), (50, 168, 82), (219, 245, 51)]

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = '/content/checkpoint_full/model_final.pth'

predictor = DefaultPredictor(cfg)

input_video_filename = '/content/drive/My Drive/data/detectron/TolucaVSPumas_sample.mp4'

input_video = cv2.VideoCapture(input_video_filename)
num_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

detections = []

i = 0
skip = 0
while(input_video.isOpened()):

  if i % 60 == 0:
    prog = round(i / num_frames, 2) * 100
    print('{}%'.format(prog))

  ret, frame = input_video.read()
  if ret == True:
    
    outputs = predictor(frame)
    pred_boxes = outputs["instances"].to("cpu").pred_boxes
    pred_boxes = pred_boxes.tensor.numpy()

    scores = outputs["instances"].to("cpu").scores
    scores = scores.numpy()

    pred_classes = outputs["instances"].to("cpu").pred_classes
    pred_classes = pred_classes.numpy()

    # pred_masks = outputs["instances"].to("cpu").pred_masks

    detections.append([i, pred_boxes, scores, pred_classes])
                   
    i += 1
  else:
    break

input_video.release()

with open('full_detections.pickle', 'wb') as f:
    pickle.dump(detections, f)

In [0]:
detections[8][3]